In [2]:
from utils import find_nearest_hit, scan_voxels_for_hits, sort_tracks
import os
import sys
import numpy as np
import matplotlib.pyplot as plt
from trackml.dataset import load_event
from trackml.score import score_event
import pdb
import pandas as pd
import csv

## Load Data ##

In [4]:
path_to_dataset = "../../Data/train_100_events/"
event_path = "event000001052"

hits, cells, particles, truth = load_event(path_to_dataset + event_path)
true_tracks = np.load("../port_toy/all_tracks.npy")

## Create Voxels ##

In [5]:
## find min/max of x,y,z ##
xMax = -sys.maxsize
yMax = -sys.maxsize
zMax = -sys.maxsize
xMin = sys.maxsize
yMin = sys.maxsize
zMin = sys.maxsize
for track in true_tracks:
    for hit in track:
        if (xMax < hit[2]): xMax = hit[2]
        if (yMax < hit[3]): yMax = hit[3]
        if (zMax < hit[4]): zMax = hit[4]
        if (xMin > hit[2]): xMin = hit[2]
        if (yMin > hit[3]): yMin = hit[3]
        if (zMin > hit[4]): zMin = hit[4]

## creating voxels ##
hits = np.asarray(hits)
xRange = xMax - xMin
yRange = yMax - yMin
zRange = zMax - zMin
n = 150
voxels = np.zeros((n+1,n+1,n+1), dtype=object)

for hit in hits:
    xHit = hit[1]
    yHit = hit[2]
    zHit = hit[3]
    i = int(n * ((xHit - xMin) / xRange))
    j = int(n * ((yHit - yMin) / yRange))
    k = int(n * ((zHit - zMin) / zRange))
    if voxels[i][j][k] == 0:
        voxels[i][j][k] = []
    voxels[i][j][k].append(hit)

print("finished creating voxels")

finished creating voxels


## Load Seeds ##

In [93]:
seed_file = open("SeedCandidates.txt", "r")
our_tracks = []
seed_hits = []
for seed_id in seed_file:
    seed_id = int(float(seed_id.strip()))
    seed_hit = hits[hits[:,0] == seed_id][0]
    our_tracks.append([int(seed_hit[0])])
    seed_hits.append(seed_hit)

# print(our_tracks)
print("starting with " + str(len(seed_hits)) + " seed hits")

starting with 9146 seed hits


## Evaluate Predicted Seeds ##

In [7]:
true_seed_ids = []
for track in true_tracks:
    true_seed_ids.append(track[0][0])

seed_ids = []
for seed_hit in seed_hits:
    seed_ids.append(seed_hit[0])

found_seeds = np.isin(seed_ids, true_seed_ids)
num_seeds_found = np.count_nonzero(found_seeds)
num_seeds_guessed = len(seed_hits)
num_real_seeds = len(true_seed_ids)

print(num_seeds_found, "/", num_real_seeds, " seeds found with", num_seeds_guessed, "predicted.")
print("recall", num_seeds_found / num_real_seeds)
print("precision", num_seeds_found / num_seeds_guessed)

5446 / 7842  seeds found with 9146 predicted.
recall 0.6944656975261413
precision 0.5954515635250383


## Import Model ##

In [8]:
## predict the next point ##
import tensorflow as tensorflow
import keras
from keras.models import load_model

## Import the stopping model
# stopping_model = load_model("final_classifier.keras")

model = load_model("3in_3out.keras")
print(model.summary())

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 500)               1008000   
_________________________________________________________________
dense_1 (Dense)              (None, 3)                 1503      
_________________________________________________________________
activation_1 (Activation)    (None, 3)                 0         
Total params: 1,009,503
Trainable params: 1,009,503
Non-trainable params: 0
_________________________________________________________________
None


## Predict 2nd Hit from True Seeds ##

In [9]:
true_seed_hits = []
for track in true_tracks:
    true_seed_hits.append(track[0])

x = []
for seed_hit in true_seed_hits:
    input_vector = np.zeros((18, 3))
    input_vector[17] = seed_hit[1:4]
    x.append(input_vector)

x = np.asarray(x)
print(x.shape)

(7842, 18, 3)


In [10]:
y = model.predict(x)
print("shape of predictions", y.shape)
print("finished predicting 2nd hits")

print(np.unique(y, axis=0))

# for line in y:
#     print(line)

shape of predictions (7842, 3)
finished predicting 2nd hits
[[708.31995  -71.86611  -63.708   ]
 [708.32     -71.8661   -63.708027]]


In [11]:
## for each prediction, find the closest hit to it ##
second_hits = []
counter = 0
for guess in y:    
    xHit = guess[0]
    yHit = guess[1]
    zHit = guess[2]
    i = int(n * ((xHit - xMin) / xRange))
    j = int(n * ((yHit - yMin) / yRange))
    k = int(n * ((zHit - zMin) / zRange))
    
    possible_nearest_hits = scan_voxels_for_hits(voxels, n, i, j, k)
    second_hit = find_nearest_hit(possible_nearest_hits, guess)
    second_hits.append(second_hit)
    if (counter % 5000) == 0:
        print(str(counter) + "/" + str(len(y)))
        print(possible_nearest_hits.shape)
    counter += 1

print("finished finding closest hits to predictions")

0/7842
(17, 7)
5000/7842
(17, 7)
finished finding closest hits to predictions


## Evaluate Predicted 2nd Hits ##

In [96]:
true_second_ids = []
for track in true_tracks:
    if (len(track) > 2): true_second_ids.append(track[1][0])

second_ids = []
for second_hit in second_hits:
    second_ids.append(second_hit[0])

print("unique second IDs found:", np.unique(np.asarray(second_ids)).shape[0])
# print(true_second_ids)
    
found_seconds = np.isin(second_ids, true_second_ids)
num_seconds_found = np.count_nonzero(found_seconds)
num_seconds_guessed = len(second_hits)
num_real_seconds = len(true_second_ids)

print(num_seconds_found, "/", num_real_seconds, " second hits found with", num_seconds_guessed, "predicted.")
print("recall", num_seconds_found / num_real_seconds)
print("precision", num_seconds_found / num_seconds_guessed)

[16373.0, 43981.0, 39443.0, 39164.0, 39337.0, 24320.0, 17788.0, 23390.0, 23530.0, 23522.0, 37801.0, 24646.0, 61800.0, 24696.0, 18823.0, 29056.0, 23937.0, 23062.0, 23945.0, 15607.0, 28641.0, 29349.0, 17851.0, 24991.0, 24977.0, 12506.0, 12494.0, 13260.0, 14124.0, 22636.0, 8460.0, 11315.0, 13026.0, 58785.0, 23525.0, 22573.0, 23338.0, 21488.0, 15312.0, 8115.0, 22233.0, 22263.0, 12953.0, 13029.0, 11242.0, 11528.0, 13840.0, 11364.0, 11349.0, 13881.0, 13414.0, 11286.0, 22006.0, 11818.0, 11710.0, 12268.0, 15294.0, 25948.0, 31644.0, 24185.0, 26947.0, 7505.0, 12490.0, 21039.0, 26627.0, 13486.0, 20928.0, 25023.0, 39018.0, 42333.0, 42077.0, 68764.0, 69074.0, 69151.0, 42177.0, 39223.0, 39001.0, 17824.0, 16425.0, 23456.0, 26269.0, 29904.0, 24193.0, 15752.0, 15901.0, 23234.0, 24285.0, 25618.0, 31058.0, 24972.0, 24954.0, 19115.0, 12723.0, 21293.0, 10568.0, 13417.0, 21976.0, 21664.0, 22320.0, 27716.0, 22539.0, 28368.0, 23460.0, 22599.0, 22087.0, 22018.0, 22554.0, 22258.0, 21576.0, 13543.0, 11476.0, 883

## Predict and Evaluate 3rd Hit from True (1st, 2nd) sequences ##

In [87]:
x = []
true_hits = []
max_len = 18
i = 1
for track in true_tracks:
#     for i in range(1, max_len):
        x_hit = np.zeros((max_len, 3))

        if i < len(track)-1:
            for z in range(i):
                x_hit[max_len-i+z] = track[z][2:5]
            x.append(x_hit)
            true_hits.append(track[i+1])
            
x = np.asarray(x)
print(x.shape)
# print(x[0])

(7400, 18, 3)


In [88]:
y = model.predict(x)
print("shape of predictions", y.shape)
print("finished predicting hits")

shape of predictions (7400, 3)
finished predicting hits


In [89]:
predicted_hits = []
counter = 0
for guess in y:    
    xHit = guess[0]
    yHit = guess[1]
    zHit = guess[2]
    ii = int(n * ((xHit - xMin) / xRange))
    j = int(n * ((yHit - yMin) / yRange))
    k = int(n * ((zHit - zMin) / zRange))
    
    possible_nearest_hits = scan_voxels_for_hits(voxels, n, ii, j, k)
    hit = find_nearest_hit(possible_nearest_hits, guess)
    predicted_hits.append(hit)
    if (counter % 5000) == 0:
        print(str(counter) + "/" + str(len(y)))
        print(possible_nearest_hits.shape)
    counter += 1

print("finished finding closest hits to predictions")
predicted_hits = np.asarray(predicted_hits)
print(predicted_hits.shape)

0/7400
(69, 7)
5000/7400
(45, 7)
finished finding closest hits to predictions
(7400, 7)


In [92]:
true_ids = []
for hit in true_hits:
    true_ids.append(hit[0])
true_ids = np.asarray(true_ids)

predicted_ids = []
for hit in predicted_hits:
    predicted_ids.append(hit[0])
predicted_ids = np.asarray(predicted_ids)

found = np.equal(predicted_ids, true_ids)

num_true_found = np.count_nonzero(found)
num_true = len(true_ids)

print("%.4f%% accuracy for hit #%d" % (num_true_found / num_true, i+1))

0.0080% accuracy for hit #2
